# 2023 CITS4012 Assignment
*Make sure you change the file name with your student id.*

# Readme
*If there is something to be noted for the marker, please mention here.* 

*If you are planning to implement a program with Object Oriented Programming style, please check the bottom of the this ipynb file*

In [2]:
import sys
sys.path.append('./src/QA')
from importlib import reload
from data_prep import DataPrep
reload(data_prep)
from data_prep import DataPrep

NameError: name 'data_prep' is not defined

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

In [ ]:
# only need to run these commands if using Colab
# Mount google drive and change directory to the git repo folder
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/CITS4012_2023

In [ ]:
# colab seems to require an explicit path for this to work from mounted drive
import sys
sys.path.append('/content/gdrive/MyDrive/CITS4012_2023/src/QA')
from data_prep import DataPrep

In [6]:
# when not using colab
import sys
sys.path.append('./src/QA')
from data_prep import DataPrep

In [7]:
df = DataPrep.parse_tsv('WikiQA-train.tsv')
question_id_list = df.loc[:, "QuestionID"]
question_id_list = list(set(question_id_list))
question_id_list.sort() # fix sorting or dont sort

question_id = {}
for i in df.index:
    row = df.loc[i]
    id = row["QuestionID"]
    if id not in question_id:
        question_id[id] = {"document": []}
        question_id[id]["question"] = row["Question"]
    
    question_id[id]["document"].append((row["Sentence"], row["Label"]))

In [8]:
toks = DataPrep.tokenize_question_and_doc(question_id["Q1"])
toks

(['how', 'are', 'glacier', 'caves', 'formed?'],
 [[['A', 'OOA'],
   ['partly', 'OOA'],
   ['submerged', 'OOA'],
   ['glacier', 'OOA'],
   ['cave', 'OOA'],
   ['on', 'OOA'],
   ['Perito', 'OOA'],
   ['Moreno', 'OOA'],
   ['Glacier', 'OOA'],
   ['.', 'OOA']],
  [['The', 'OOA'],
   ['ice', 'OOA'],
   ['facade', 'OOA'],
   ['is', 'OOA'],
   ['approximately', 'OOA'],
   ['60', 'OOA'],
   ['m', 'OOA'],
   ['high', 'OOA']],
  [['Ice', 'OOA'],
   ['formations', 'OOA'],
   ['in', 'OOA'],
   ['the', 'OOA'],
   ['Titlis', 'OOA'],
   ['glacier', 'OOA'],
   ['cave', 'OOA']],
  [['A', 'BOA'],
   ['glacier', 'IOA'],
   ['cave', 'IOA'],
   ['is', 'IOA'],
   ['a', 'IOA'],
   ['cave', 'IOA'],
   ['formed', 'IOA'],
   ['within', 'IOA'],
   ['the', 'IOA'],
   ['ice', 'IOA'],
   ['of', 'IOA'],
   ['a', 'IOA'],
   ['glacier', 'IOA'],
   ['.', 'EOA']],
  [['Glacier', 'OOA'],
   ['caves', 'OOA'],
   ['are', 'OOA'],
   ['often', 'OOA'],
   ['called', 'OOA'],
   ['ice', 'OOA'],
   ['caves', 'OOA'],
   [',', 'OO

# 2.QA Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

# 3.Model Testing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

###3.1. Input Embedding Ablation Study

(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 



###3.2. Attention Ablation Study
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

###3.3. Hyper Parameter Testing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 